In [104]:
# %pip install git+https://github.com/tensorflow/docs

import requests
import os.path
import asyncio
import json
import numpy as np
import pathlib
import pandas as pd
import seaborn as sns
import tensorflow as tf
import time
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling
from riotwatcher import TftWatcher, ApiError

from utils.configuration import settings
from utils.utils import *
from tft.api import *


In [105]:
API_KEY = settings.api_key
ASSETS_DIR = settings.assets_dir
SERVER = 'kr'  # euw1 na1 kr oc1
LEAGUE='challengers'
MAX_COUNT = settings.max_count
LATEST_RELEASE = '12.12.450.4196' # '12.12.450.4196' Version 12.12.448.6653 12.11.446.9344

In [106]:
summoners_df = pd.read_pickle(os.path.join(ASSETS_DIR, f'{SERVER}_{LEAGUE}_summoners.pickle'))

In [107]:
summoners_df

,id,accountId,puuid,name,profileIconId,revisionDate,summonerLevel,summonerId,summonerName,leaguePoints,rank,wins,losses,veteran,inactive,freshBlood,hotStreak
0,cf_qi753SihFOmDNUwUNYerSmIKvp8PmVYhUnmDEAi5QbV8,0a9ChyOhT4Z91XfXdIf-4xWCHg3s1icdms6WkPuKD8UPdqA,dyFAaW_H8PP9KQkoYh4j3BLm58s3CQRyaS2uoAkRvHLVvE...,TwTF 근열맨,29,1656433956000,735,cf_qi753SihFOmDNUwUNYerSmIKvp8PmVYhUnmDEAi5QbV8,TwTF 근열맨,724,I,171,122,False,False,True,False
1,GWi0DLb_Ff7aVatJRcazhDAusFsddNsDJvZsN1aOCW83HxM,EdmiLw0wCWaMmUuE2nCu7lYGwwWPevSbxX8u9K467ZClrLE,AWi4QF9Qi8FCwu0HTa2cOHTfaWSgqarHzw1STwik4JUuN8...,와라이오니,5271,1656443241021,458,GWi0DLb_Ff7aVatJRcazhDAusFsddNsDJvZsN1aOCW83HxM,와라이오니,520,I,122,80,False,False,True,False
2,JKP26SMAVLk4h4QqoLGO16rnWCvaCFqDrLtvhnvqeRr4q-Ix,-yvFmc0YwKgnG-AKKfkUZSIqMuq198_p_NM-lf_91pr-W7...,mC4V4DcaoHdSgy3Kb58mzh3FB-f7MOB1H9ICWHCshXgSbl...,김강한2,5371,1656122320000,3,JKP26SMAVLk4h4QqoLGO16rnWCvaCFqDrLtvhnvqeRr4q-Ix,김강한2,590,I,153,113,False,False,True,True
3,6LIq4MiaZS0tqvKFOunlilefwad8Rh3w3RpHj_wGCF9OaJ...,35VYluO8zYnRd0CNOBWRr_vO6PL4TTiswJPsO40MBXOKAl...,aOgC7Bdbb2B_uI3iB_sOp5yZ1jTCvQ1koXL5vyWOyDYoI1...,God Gyeom,29,1656087381000,1,6LIq4MiaZS0tqvKFOunlilefwad8Rh3w3RpHj_wGCF9OaJ...,God Gyeom,605,I,73,55,False,False,True,False
4,iU954V7Ds5cAQkNU8iFm7krNeGk6MWvRWb0upctmEZulnA,Es2fdrnBno93bHkG7RDEjOYIf7PNEEGrCrDnA_E8o-c,D1rgzM_eZOr7zl6CpDLJ1e76T68vjJC7lDtulC32XGtPp9...,퀸나르,4928,1656407683000,364,iU954V7Ds5cAQkNU8iFm7krNeGk6MWvRWb0upctmEZulnA,퀸나르,511,I,136,109,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116,Nngx-dcb64k4laBUMCgaRDz5ZRoILnzG76d8zduxtZRpDA,rb2KNi2V0shkm5j2d_qtiLhPDlZUpAzwClSePCZkI5Lh,2zHwCsblDVEjoPemlRh3dhNRGsxDY3oY8Icw6w7vX0Mv3y...,호 까,4568,1656266475000,217,Nngx-dcb64k4laBUMCgaRDz5ZRoILnzG76d8zduxtZRpDA,호 까,509,I,116,89,False,False,True,False
117,5-eSuW3C-39OMFTxJiDrj64KUtjMzzqJuBHYXSIxkajVCaE,TWB-XWrDL52MebgCvpZKf1p0P8rpnb9akljPBR90-O2A7i...,ktrxQQiWlEMf4zo7dzOKzKdKzbHNdoShinADqP1hrpc7c5...,므네짱짱,4274,1656468786000,2,5-eSuW3C-39OMFTxJiDrj64KUtjMzzqJuBHYXSIxkajVCaE,므네짱짱,807,I,119,97,False,False,True,False
118,Y3Z5DdqNn4ydbxaXJovTlhZz8kQVJZH0T-SvTjPWYb-Daw,9QtoRX3Izs4j6Ic0o160hMlFL-PIu9NkJk0A8hNWqp8x,TMXYfV_LvjGLfMqqDmF2adhc2C2-Z7YTkWXlyhvtokIw86...,Hwj,4419,1656441508000,317,Y3Z5DdqNn4ydbxaXJovTlhZz8kQVJZH0T-SvTjPWYb-Daw,Hwj,556,I,162,126,False,False,True,False
119,Y99KBNJIEqW0rblSMeJibH4iM0DMZccMqJEYaF83XNuMcKg,rng4L_9quiDxiRVNK-HzbTsERX5HeKQXlIdPHYvGrIvItjg,hsuYXDf4eJrysAn9D2h-r5-Ze-rGGGh8YSoAXr7xoHsdSC...,a hickey,5091,1656442811000,282,Y99KBNJIEqW0rblSMeJibH4iM0DMZccMqJEYaF83XNuMcKg,a hickey,854,I,139,107,False,False,True,False


In [108]:

# Get all unique matches_id from assets dir
matches_asset = load_matches(summoners_df, server=SERVER)
matches_id = [match['metadata']['match_id'] for match in matches_asset]
seen = set()
uniq_matches_id = [
    x for x in matches_id if x not in seen and not seen.add(x)]

In [109]:
len(uniq_matches_id)

1951

In [110]:
len(matches_asset)

2646

In [111]:
seen = set()
seen_add = seen.add
uniq_matches = [x for x in matches_asset if x['metadata']['match_id'] not in seen and not seen_add(x['metadata']['match_id'])]

In [112]:
len(uniq_matches)

1951

In [113]:
matches_asset[300]['info']['game_version']

'Version 12.12.450.4196 (Jun 24 2022/13:37:31) [PUBLIC] <Releases/12.12>'

In [114]:
latest_matches = [match for match in uniq_matches if LATEST_RELEASE in match['info']['game_version']]

In [115]:
len(latest_matches)

1275

In [116]:
# latest_matches[0]['info']

In [117]:
matches_array = []

for match_row in latest_matches:
    match_id = match_row['metadata']['match_id']

    for participant in match_row['info']['participants']:
        match = {}
        match['match_id'] = match_id
        # match['level'] = participant['level']
        match['placement'] = participant['placement']
        # match['players_eliminated'] = participant['players_eliminated']
        # match['total_damage_to_players'] = participant['total_damage_to_players']

        for augment_index, augment in enumerate(participant['augments']):
            if augment == 'TFT7_Augment_GuildLootHR':
                augment = 'TFT7_Augment_BandOfThieves1'
            match[f'augment{augment_index}'] = augment

        for trait_index, trait in enumerate(participant['traits']):
            match[f'{trait["name"]}'] = trait["tier_current"]

        for unit_index, unit in enumerate(participant['units']):
            match[f'{unit["character_id"]}'] = unit["tier"]
            for item_index, item in enumerate(unit['itemNames']):
                match[f'{unit["character_id"]}_item{item_index}'] = item

        matches_array.append(match)

In [123]:
matches_array[0]
# len(matches_array)

{'match_id': 'KR_5992522314',
 'placement': 5,
 'augment0': 'TFT6_Augment_TradeSector',
 'augment1': 'TFT7_Augment_BigFriend',
 'augment2': 'TFT6_Augment_RadiantRelics',
 'Set7_Bruiser': 0,
 'Set7_Dragon': 1,
 'Set7_Dragonmancer': 1,
 'Set7_Evoker': 0,
 'Set7_Jade': 2,
 'Set7_Legend': 1,
 'Set7_Mystic': 0,
 'Set7_Shapeshifter': 1,
 'Set7_Shimmerscale': 0,
 'Set7_Tempest': 1,
 'TFT7_Gnar': 2,
 'TFT7_LeeSin': 2,
 'TFT7_Volibear': 2,
 'TFT7_Volibear_item0': 'TFT_Item_GuinsoosRageblade',
 'TFT7_Volibear_item1': 'TFT_Item_MadredsBloodrazor',
 'TFT7_Volibear_item2': 'TFT5_Item_RapidFirecannonRadiant',
 'TFT7_Anivia': 2,
 'TFT7_Neeko': 2,
 'TFT7_Neeko_item0': 'TFT_Item_IonicSpark',
 'TFT7_Neeko_item1': 'TFT_Item_BrambleVest',
 'TFT7_Neeko_item2': 'TFT_Item_Redemption',
 'TFT7_Ornn': 2,
 'TFT7_Ornn_item0': 'TFT_Item_Morellonomicon',
 'TFT7_Ornn_item1': 'TFT7_Item_DragonmancerEmblemItem',
 'TFT7_Ornn_item2': 'TFT_Item_ArchangelsStaff',
 'TFT7_DragonGreen': 2,
 'TFT7_DragonGreen_item0': 'TFT_Ite

In [119]:
matches_league_df = pd.json_normalize(matches_array)

In [120]:
matches_league_df

,match_id,placement,augment0,augment1,augment2,Set7_Bruiser,Set7_Cannoneer,Set7_Dragon,Set7_Evoker,Set7_Guardian,...,TFT7_Heimerdinger_item1,TFT7_Heimerdinger_item2,TFT7_Skarner_item1,TFT7_Senna_item0,TFT7_Sett_item2,TFT7_Aatrox_item1,TFT7_Aatrox_item2,TFT7_Senna_item1,TFT7_Senna_item2,TFT7_Skarner_item2
0,KR_5992522314,6,TFT6_Augment_Ascension,TFT6_Augment_TinyTitans,TFT6_Augment_Diversify3,1.0,1.0,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,KR_5992522314,5,TFT6_Augment_TradeSector,TFT7_Augment_BigFriend,TFT6_Augment_RadiantRelics,0.0,NaN,1.0,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,KR_5992522314,2,TFT6_Augment_RichGetRicher,TFT6_Augment_MakeshiftArmor1,TFT6_Augment_WindfallPlusPlus,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,KR_5992522314,3,TFT7_Augment_MikaelsGift,TFT6_Augment_Weakspot,TFT6_Augment_RadiantRelics,NaN,2.0,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,KR_5992522314,1,TFT6_Augment_ThrillOfTheHunt2,TFT7_Augment_Preparation,TFT7_Augment_CavalierEmblem2,0.0,NaN,NaN,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10195,KR_5990829538,5,TFT6_Augment_RichGetRicher,TFT7_Augment_DragonmancerInspire,TFT7_Augment_DragonmancerEmblem,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10196,KR_5990829538,1,TFT7_Augment_CannoneerHotShot,TFT6_Augment_TargetDummies,TFT7_Augment_RevelPartyTime,NaN,1.0,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10197,KR_5990829538,2,TFT6_Augment_ThrillOfTheHunt2,TFT7_Augment_MirageHallucinate,TFT7_Augment_UrfsGrabBag1,1.0,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10198,KR_5990829538,3,TFT6_Augment_SalvageBin,TFT6_Augment_ThrillOfTheHunt2,TFT6_Augment_TomeOfTraits1,1.0,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [121]:
matches_league_df[matches_league_df.isnull().T.any()]

,match_id,placement,augment0,augment1,augment2,Set7_Bruiser,Set7_Cannoneer,Set7_Dragon,Set7_Evoker,Set7_Guardian,...,TFT7_Heimerdinger_item1,TFT7_Heimerdinger_item2,TFT7_Skarner_item1,TFT7_Senna_item0,TFT7_Sett_item2,TFT7_Aatrox_item1,TFT7_Aatrox_item2,TFT7_Senna_item1,TFT7_Senna_item2,TFT7_Skarner_item2
0,KR_5992522314,6,TFT6_Augment_Ascension,TFT6_Augment_TinyTitans,TFT6_Augment_Diversify3,1.0,1.0,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,KR_5992522314,5,TFT6_Augment_TradeSector,TFT7_Augment_BigFriend,TFT6_Augment_RadiantRelics,0.0,NaN,1.0,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,KR_5992522314,2,TFT6_Augment_RichGetRicher,TFT6_Augment_MakeshiftArmor1,TFT6_Augment_WindfallPlusPlus,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,KR_5992522314,3,TFT7_Augment_MikaelsGift,TFT6_Augment_Weakspot,TFT6_Augment_RadiantRelics,NaN,2.0,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,KR_5992522314,1,TFT6_Augment_ThrillOfTheHunt2,TFT7_Augment_Preparation,TFT7_Augment_CavalierEmblem2,0.0,NaN,NaN,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10195,KR_5990829538,5,TFT6_Augment_RichGetRicher,TFT7_Augment_DragonmancerInspire,TFT7_Augment_DragonmancerEmblem,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10196,KR_5990829538,1,TFT7_Augment_CannoneerHotShot,TFT6_Augment_TargetDummies,TFT7_Augment_RevelPartyTime,NaN,1.0,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10197,KR_5990829538,2,TFT6_Augment_ThrillOfTheHunt2,TFT7_Augment_MirageHallucinate,TFT7_Augment_UrfsGrabBag1,1.0,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10198,KR_5990829538,3,TFT6_Augment_SalvageBin,TFT6_Augment_ThrillOfTheHunt2,TFT6_Augment_TomeOfTraits1,1.0,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [122]:
matches_league_df.to_pickle(os.path.join(ASSETS_DIR, f'{SERVER}_{LEAGUE}_{LATEST_RELEASE}_matches.pickle'))